In [30]:
import pandas as pd
import numpy as np 
import scipy as sc
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
import plotly.express as px
import plotly.graph_objects as go
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed

In [3]:
# Used to read data from S3 bucket
import boto3
import s3fs
import pandas as pd
from sagemaker import get_execution_role

In [4]:
colum=['DATE', 'VIN', 'IIN', 'VOUT', 'VBATT', 'IBATT', 'TEMP', 'JP', 'CNT','STS', 'MIN', 'POWER', 'AUTOBCKLIGHT', 'CURRLIGHT', 'BRIGHTNESS',
       'PAN1VOLT', 'PAN1CURR', 'BCK1VOLT', 'BCK1CURR', 'RSSI', 'Batt_id']
my_Data=pd.DataFrame(columns=colum)
my_Data

,DATE,VIN,IIN,VOUT,VBATT,IBATT,TEMP,JP,CNT,STS,...,POWER,AUTOBCKLIGHT,CURRLIGHT,BRIGHTNESS,PAN1VOLT,PAN1CURR,BCK1VOLT,BCK1CURR,RSSI,Batt_id


In [5]:
role = get_execution_role()
fs = s3fs.S3FileSystem()
list_file=fs.ls('s3://datanovember/Giorni_November/')[:6]
list_file=[x.split('/')[2] for x in list_file]
# list_file

In [6]:
bucket='s3://datanovember'
for i , file in enumerate(list_file): 
    data_key = 'Giorni_November/'+file
    data_location = 's3://{}/{}'.format(bucket, data_key)
    df=pd.read_csv(data_location, sep=';')
    my_Data=my_Data.append(df,ignore_index=True)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
my_Data.head(4)

,DATE,VIN,IIN,VOUT,VBATT,IBATT,TEMP,JP,CNT,STS,...,AUTOBCKLIGHT,CURRLIGHT,BRIGHTNESS,PAN1VOLT,PAN1CURR,BCK1VOLT,BCK1CURR,RSSI,Batt_id,Unnamed: 0
0,2020-10-28 00:00:55,29168,625.0,29139.0,29068.0,663.0,245.0,OFF,8119.0,1.0,...,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-79.0,814U01M70006,0.0
1,2020-10-28 00:01:55,29170,665.0,29139.0,29063.0,558.0,245.0,OFF,8120.0,1.0,...,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-79.0,814U01M70006,1.0
2,2020-10-28 00:02:54,29167,632.0,29139.0,29066.0,564.0,245.0,OFF,8121.0,1.0,...,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-79.0,814U01M70006,2.0
3,2020-10-28 00:03:53,29172,619.0,29139.0,29058.0,535.0,245.0,OFF,8122.0,1.0,...,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-79.0,814U01M70006,3.0


In [8]:
my_Data["VIN"] = my_Data["VIN"].fillna(my_Data["VIN"].mean())
my_Data["VIN"]=my_Data["VIN"].astype(int)
my_Data["IIN"] = my_Data["IIN"].fillna(my_Data["IIN"].mean())
my_Data["IIN"]=my_Data["IIN"].astype(int)
my_Data["VOUT"] = my_Data["VOUT"].fillna(my_Data["VOUT"].mean())
my_Data["VOUT"]=my_Data["VOUT"].astype(int)
my_Data["VBATT"] = my_Data["VBATT"].fillna(my_Data["VBATT"].mean())
my_Data["VBATT"]=my_Data["VBATT"].astype(int)
my_Data["IBATT"] = my_Data["IBATT"].fillna(my_Data["IBATT"].mean())
my_Data["IBATT"]=my_Data["IBATT"].astype(int)
my_Data["TEMP"] = my_Data["TEMP"].fillna(my_Data["TEMP"].mean())
my_Data["TEMP"]=my_Data["TEMP"].astype(int)
my_Data["CNT"] = my_Data["CNT"].fillna(my_Data["CNT"].mean())
my_Data["CNT"]=my_Data["CNT"].astype(int)
my_Data["POWER"] = my_Data["POWER"].fillna(my_Data["POWER"].mean())
my_Data["POWER"]=my_Data["POWER"].astype(int)

## LSTM

In [13]:

New_DF=my_Data[my_Data.Batt_id=='814U01M70988']
New_DF

,DATE,VIN,IIN,VOUT,VBATT,IBATT,TEMP,JP,CNT,STS,...,AUTOBCKLIGHT,CURRLIGHT,BRIGHTNESS,PAN1VOLT,PAN1CURR,BCK1VOLT,BCK1CURR,RSSI,Batt_id,Unnamed: 0
903398,2020-10-28 00:00:12,29136,647,29097,29032,587,240,OFF,91549,1.0,...,1,12.0,0.0,11896.0,490.0,23992.0,389.0,-67.0,814U01M70988,903398.0
903399,2020-10-28 00:01:11,29136,663,29097,29038,635,240,OFF,91550,1.0,...,1,12.0,0.0,11896.0,485.0,23996.0,389.0,-67.0,814U01M70988,903399.0
903400,2020-10-28 00:02:10,29140,625,29097,29036,644,240,OFF,91551,1.0,...,1,12.0,0.0,11896.0,485.0,23996.0,389.0,-67.0,814U01M70988,903400.0
903401,2020-10-28 00:03:09,29135,648,29097,29041,520,240,OFF,91552,1.0,...,1,12.0,6.0,11896.0,490.0,23996.0,394.0,-67.0,814U01M70988,903401.0
903402,2020-10-28 00:04:08,29136,619,29097,29037,572,240,OFF,91553,1.0,...,1,12.0,6.0,11896.0,490.0,23992.0,394.0,-67.0,814U01M70988,903402.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5926090,2020-11-03 23:55:38,29138,619,29097,29042,583,190,OFF,101773,1.0,...,1,12.0,6.0,11900.0,490.0,23992.0,394.0,-69.0,814U01M70988,896984.0
5926091,2020-11-03 23:56:37,29140,627,29097,29037,554,190,OFF,101774,1.0,...,1,12.0,6.0,11900.0,490.0,23992.0,394.0,-69.0,814U01M70988,896985.0
5926092,2020-11-03 23:57:36,29137,637,29097,29042,528,190,OFF,101775,1.0,...,1,12.0,6.0,11900.0,490.0,23992.0,394.0,-69.0,814U01M70988,896986.0
5926093,2020-11-03 23:58:36,29140,644,29097,29040,538,190,OFF,101776,1.0,...,1,12.0,6.0,11900.0,490.0,23992.0,394.0,-69.0,814U01M70988,896987.0


In [14]:
New_DF = New_DF.rename({'Batt_id': 'host_id'}, axis='columns')
New_DF['time']=[x.split(' ')[1] for x in New_DF['DATE']]
New_DF['date']=[x.split(' ')[1] for x in New_DF['DATE']]

New_DF['time']=[datetime.datetime.strptime(x,'%H:%M:%S').time() for x in New_DF['time']]
New_DF['time_sec']=[sum([x.hour*3600,x.minute*60,x.second]) for x in New_DF['time']]
### Code for host ID 
New_DF.host_id = pd.Categorical(New_DF.host_id)
New_DF['code_host_id'] = New_DF.host_id.cat.codes

In [15]:
New_DF=New_DF[['VIN', 'IIN', 'VOUT', 'VBATT', 'IBATT', 'TEMP', 'CNT',  'POWER','time_sec']]

# Step 1 :splitting the dataset into training and testing sets

In [16]:
train_size = int(New_DF.shape[0] * 0.8)
test_size = New_DF.shape[0] - train_size
train, test = New_DF.iloc[0:train_size], New_DF.iloc[train_size:New_DF.shape[0]]
print(train.shape, test.shape)

(7014, 9) (1754, 9)


# Step 2: Data standardization and scaling

## 2.1- For training data

In [19]:
scaler = StandardScaler()
scaler = scaler.fit(train[['VBATT']])### we fit the normalizer on training data 
train['VBATT']=scaler.transform(train[['VBATT']])
train['VBATT']

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


903398     0.952799
903399     0.957571
903400     0.955980
903401     0.959956
903402     0.956775
             ...   
4930717    0.952799
4930718    0.955980
4930719    0.955185
4930720    0.952799
4930721    0.951209
Name: VBATT, Length: 7014, dtype: float64

## 2.2- For testing data

In [20]:
test['VBATT'] = scaler.transform(test[['VBATT']])
test['VBATT']

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


4930722    0.955185
4930723    0.955185
4930724    0.955185
4930725    0.952004
4930726    0.951209
             ...   
5926090    0.960751
5926091    0.956775
5926092    0.960751
5926093    0.959161
5926094    0.959161
Name: VBATT, Length: 1754, dtype: float64

# Step 3: Build an LSTM Autoencoder

In [21]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [34]:
time_steps = 1461
X_train, y_train = create_dataset(train[['VBATT']], train.VBATT, time_steps)
X_test, y_test = create_dataset(test[['VBATT']], test.VBATT, time_steps)
print(X_train.shape)

(5553, 1461, 1)


In [35]:
timesteps = X_train.shape[1]
num_features = X_train.shape[2]

In [36]:
model = Sequential([
    LSTM(32, input_shape=(timesteps, num_features)),
    Dropout(0.2),
    RepeatVector(timesteps),
    LSTM(32, return_sequences=True),
    Dropout(0.2),
    TimeDistributed(Dense(num_features))                 
])

model.compile(loss='mae', optimizer='adam')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 1461, 32)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 1461, 32)          8320      
_________________________________________________________________
dropout_5 (Dropout)          (None, 1461, 32)          0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 1461, 1)           33        
Total params: 12,705
Trainable params: 12,705
Non-trainable params: 0
__________________________________________________

# Step 4: Train the Autoencoder

In [39]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min')
history = model.fit( X_train, y_train,epochs=100,batch_size=8, validation_split=0.1,callbacks = [es],shuffle=False)

Train on 4997 samples, validate on 556 samples
Epoch 1/100


InvalidArgumentError: Incompatible shapes: [8,1461,1] vs. [8,1]
	 [[{{node loss_2/time_distributed_2_loss/sub}}]]

In [38]:
X_train.shape

(5553, 1461, 1)